In [ ]:
import dspy
from src.roma_dspy.modules import Executor, Atomizer, Planner, Aggregator
from src.roma_dspy.engine.solve_refactored import RecursiveSolver
from src.roma_dspy.engine.visualizer import ExecutionVisualizer

executor_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
atomizer_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
planner_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
aggregator_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)

# Initialize modules
atomizer = Atomizer(lm=atomizer_lm)
planner = Planner(lm=planner_lm)
executor = Executor(lm=executor_lm)
aggregator = Aggregator(lm=aggregator_lm)

# Create solver
solver = RecursiveSolver(
    atomizer,
    planner,
    executor,
    aggregator,
    max_depth=2
)

In [2]:
# Test task - use a simpler task for clearer output
task_goal = "Write me a blog post about the benefits of using DSPy."

print("=" * 80)
print("ROMA-DSPy Detailed Execution Visualization")
print("=" * 80)
print(f"\n🎯 Goal: {task_goal}\n")

# Execute task
print("⚙️  Executing task...\n")
result = solver.solve(task_goal)

2025/09/23 16:19:18 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.


ROMA-DSPy Detailed Execution Visualization

🎯 Goal: Write me a blog post about the benefits of using DSPy.

⚙️  Executing task...



2025/09/23 16:19:19 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.
2025/09/23 16:19:21 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.
2025/09/23 16:19:22 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.
2025/09/23 16:19:25 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.
2025/09/23 16:19:27 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.
2025/09/23 16:19:30 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.
2025/09/23 16:19:35 WARNING dspy.primitives.module: Calling module.for

In [3]:
# Get the DAG for detailed visualization
dag = solver.last_dag

if dag:
    # Show the full execution report
    visualizer = ExecutionVisualizer()
    print(visualizer.get_full_execution_report(result, dag))

    # Also show the tree view with details
    print("\n" + "=" * 80)
    print("🌳 EXECUTION TREE WITH DETAILS")
    print("=" * 80)
    print(visualizer.get_execution_tree_with_details(result, dag))
else:
    print("No DAG available for visualization")

📊 COMPLETE EXECUTION REPORT

🎯 GOAL: Write me a blog post about the benefits of using DSPy.
📌 Status: COMPLETED
📐 Depth: 0/2
No planning information available.

⚡ SUBTASK EXECUTIONS

📁 NESTED PLAN: Research DSPy to understand its core concepts, functionalities, and primary benefits for LLM development.
No planning information available.

⚡ EXECUTION NODE: Find the official DSPy documentation and any high-level overview articles or videos.
--------------------------------------------------------------------------------
Duration: 2.39s

📤 Output:
Official DSPy Documentation:
- https://dspy.readthedocs.io/en/latest/

High-Level Overviews and Introductions:
- **Blog Post:** DSPy: Programming Foundation Models with a New Programming Model (https://dspy.ai/blog/dspy-programming-foundation-models-with-a-new-programming-model/)
- **Video:** DSPy Explained: The New Way to Program LLMs (https://www.youtube.com/watch?v=z_p_g0_2c0Q)
- **Video:** DSPy: The New Programming Model for LLMs (https://ww

In [6]:
print(result.result)

# Unlocking LLM Potential: Why DSPy is a Game-Changer for AI Development

## I. Introduction

Are you tired of wrestling with complex prompt engineering, struggling to get consistent and reliable outputs from your Large Language Models (LLMs)? What if there was a way to program LLMs more systematically, like traditional software, and achieve significantly better results with less effort?

Enter DSPy, a groundbreaking framework designed to optimize and program LLMs. Unlike traditional prompt engineering, which often relies on trial-and-error, DSPy treats LLMs as components within a larger program, allowing developers to compose, compile, and optimize complex LLM pipelines. It provides a systematic approach to building robust and efficient LLM-powered applications, moving beyond simple prompting to a more structured and performant paradigm.

In an era where LLMs are becoming central to countless applications, the ability to reliably and efficiently develop with them is paramount. DSPy ad